In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Votre clé API Alpha Vantage
api_key = '3UGFQBY1X5H1BVP0'
interval = "60min"  # Exemple : Données toutes les 60 minutes


# Dictionnaire des symboles et types d'actifs
assets = {
    'S&P 500': {'symbol': 'SPY', 'function': 'TIME_SERIES_INTRADAY', 'market': 'USD'},
    'Bitcoin': {'symbol': 'BTC', 'function': 'DIGITAL_CURRENCY_INTRADAY', 'market': 'USD'},
    'Or': {'symbol': 'XAU', 'function': 'TIME_SERIES_INTRADAY', 'market': 'USD'}
}

# Date de début et de fin
start_date = datetime(2024, 3, 1)
end_date = datetime.now()

# Fonction pour récupérer les données
def fetch_data(symbol, function, market, interval='60min'):
    if function == 'TIME_SERIES_INTRADAY':
        url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&interval={interval}&apikey={api_key}'
    elif function == 'DIGITAL_CURRENCY_INTRADAY':
        url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&market={market}&apikey={api_key}'
    else:
        raise ValueError("Fonction non supportée")
    response = requests.get(url)
    data = response.json()
    return data

# Parcourir chaque actif et récupérer les données
for asset_name, asset_info in assets.items():
    print(f"Téléchargement des données pour {asset_name}...")
    data = fetch_data(asset_info['symbol'], asset_info['function'], asset_info['market'])
    
    # Traitement des données en fonction du type d'actif
    if asset_info['function'] == 'TIME_SERIES_INTRADAY':
        time_series_key = f'Time Series ({interval})'
        df = pd.DataFrame.from_dict(data[time_series_key], orient='index')
        df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    elif asset_info['function'] == 'DIGITAL_CURRENCY_INTRADAY':
        time_series_key = 'Time Series Crypto (5min)'
        df = pd.DataFrame.from_dict(data[time_series_key], orient='index')
        df.columns = ['Open (USD)', 'High (USD)', 'Low (USD)', 'Close (USD)', 'Volume', 'Market Cap (USD)']
    else:
        continue

    # Conversion de l'index en datetime
    df.index = pd.to_datetime(df.index)

    # Filtrer les données entre les dates de début et de fin
    df = df[(df.index >= start_date) & (df.index <= end_date)]

    # Enregistrer les données dans un fichier CSV
    df.to_csv(f'{asset_name}_historical_data.csv')
    print(f"Les données pour {asset_name} ont été enregistrées dans {asset_name}_historical_data.csv")

print("Téléchargement terminé.")


Téléchargement des données pour S&P 500...
Les données pour S&P 500 ont été enregistrées dans S&P 500_historical_data.csv
Téléchargement des données pour Bitcoin...


KeyError: 'Time Series Crypto (5min)'